In [25]:
import torch
import torch.nn as nn
import torch.functional as F

import numpy as np

# 1.1.1 벡터와 행렬

In [26]:
# 3행2열 행렬 = 3x2 행렬 (2차원)
matrix_a = np.array([
    [1,2],
    [3,4],
    [5,6]
])
matrix_a.shape, matrix_a.ndim

((3, 2), 2)

# 1.1.2 행렬의 원소별 연산

In [27]:
# numpy의 행렬연산은 element wise로 이루어 진다.
w = np.array([
    [1,2,3],
    [4,5,6]
])
x = np.array([
    [0,1,2],
    [3,4,5]
])
w+x, w*x

(array([[ 1,  3,  5],
        [ 7,  9, 11]]),
 array([[ 0,  2,  6],
        [12, 20, 30]]))

# 1.1.3 브로드캐스트

In [28]:
# numpy의 일부 연산은 브로드캐스팅을 통해 형상이 다른 행렬 간 연산을 영리하게 수행한다.
a = np.array([
    [1,2],
    [3,4]
])
b = np.array([10,20])
a*10, a*b

(array([[10, 20],
        [30, 40]]),
 array([[10, 40],
        [30, 80]]))

# 1.1.4 벡터의 내적과 행렬의 곱

In [29]:
# numpy의 dot, matmul 은 내적과 행렬곱을 의미한다.
# dot의 경우 1차원이면 내적을, 2차원이면 행렬곱을 수행한다. 의도에 따라 혼동되지 않도록 적재적소에 활용할 것.

a = np.array([1,2,3])
b = np.array([4,5,6])

A = np.array([
    [1,2],
    [3,4]
])
B = np.array([
    [5,6],
    [7,8]
])

np.dot(a,b), np.matmul(A,B), np.dot(A,B)

(32,
 array([[19, 22],
        [43, 50]]),
 array([[19, 22],
        [43, 50]]))

### Note : 100 numpy exercises 사이트를 활용해 넘파이에 익숙해지자.

# 1.1.5 행렬 형상 확인

In [31]:
# (a,b) X (b,c) = (a,c)
A = np.ones(12).reshape(3,4)
B = (np.ones(36)*2).reshape(4,9)
A.shape, B.shape, np.matmul(A,B).shape

((3, 4), (4, 9), (3, 9))

# 1.2 신경망 추론

# 1.2.1 신경망 추론 전체 그림

In [40]:
x = np.random.randn(10,2) # 입력

w1 = np.random.randn(2,4) # 가중치
w2 = np.random.randn(4,3) # 가중치

b1 = np.random.randn(4) # 편향
b2 = np.random.randn(3)

In [41]:
h = np.matmul(x, w1)+b1 # (10,2) x (2,4) + (4,) = (10,4) (Feat : Boradcasting)

### 비선형 추가

In [48]:
sigmoid = lambda x:1/(1+np.exp(-x))
a = sigmoid(h)
a

array([[0.53491697, 0.21449872, 0.82606573, 0.58277072],
       [0.55304458, 0.41097241, 0.79907196, 0.46697806],
       [0.19716821, 0.1212743 , 0.52219402, 0.83738078],
       [0.23483369, 0.02907165, 0.67798574, 0.89818836],
       [0.20988184, 0.04515599, 0.61420929, 0.88761132],
       [0.21711748, 0.48397229, 0.42470243, 0.66907447],
       [0.68576944, 0.50582307, 0.86810626, 0.33828318],
       [0.17227321, 0.12137476, 0.47779797, 0.85140037],
       [0.54143451, 0.56499282, 0.76147599, 0.40623229],
       [0.64074915, 0.08835395, 0.90977279, 0.62547897]])

In [49]:
s = np.matmul(a, w2) + b2
s

array([[ 1.95164769, -1.3578117 , -1.20031984],
       [ 1.96582921, -1.31783951, -0.75124818],
       [ 1.20870387, -1.34757497, -1.92128665],
       [ 1.40297331, -1.43181297, -2.11690024],
       [ 1.31355344, -1.39938784, -2.09597185],
       [ 1.12337676, -1.29845856, -1.14170738],
       [ 2.19848644, -1.30120507, -0.3370698 ],
       [ 1.1269915 , -1.33851035, -1.95384532],
       [ 1.91663256, -1.30042158, -0.45500835],
       [ 2.10591097, -1.41861765, -1.33555061]])

# 1.2.2 계층으로 클래스화 및 순전파 구현
- 모든 계층은 forward(), backward() 메서드를 가진다.
- 모든 계층은 인스턴스 변수인 params, grads를 가진다.

In [51]:
# Sigmoid 는 학습변수가 없으므로 params는 비어있다.
class Sigmoid:
    def __init__(self):
        self.params = []
    def forward(self, x):
        return 1/(1+np.exp(-x))
sigmoid = Sigmoid()
sigmoid.forward(s)

array([[0.87562619, 0.20459619, 0.23141832],
       [0.87716242, 0.21117797, 0.32054939],
       [0.77006953, 0.20626712, 0.12771816],
       [0.80265528, 0.19281636, 0.10746503],
       [0.78810717, 0.19791327, 0.10948895],
       [0.75461453, 0.21442455, 0.24200702],
       [0.90011351, 0.21396228, 0.41652143],
       [0.75528326, 0.20775514, 0.12413467],
       [0.87176245, 0.21409407, 0.38817065],
       [0.89147637, 0.19487838, 0.20824271]])

In [54]:
class Affine:
    def __init__(self, W, b):
        self.params = [W,b]

    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W)+b
        return out

In [58]:
affine_1 = Affine(w1, b1)
affine_2 = Affine(w2, b2)
affine_1.forward(x), affine_2.forward(a)

(array([[ 0.13989561, -1.29801832,  1.5579969 ,  0.33415793],
        [ 0.21297977, -0.35994696,  1.38050415, -0.13228033],
        [-1.40408799, -1.98041791,  0.08883445,  1.63886748],
        [-1.18121565, -3.50848906,  0.74453041,  2.17725536],
        [-1.32563778, -3.05142511,  0.46504071,  2.06657073],
        [-1.28254406, -0.06413282, -0.30349864,  0.70400202],
        [ 0.78041451,  0.02329331,  1.88431756, -0.67095432],
        [-1.56960147, -1.9794755 , -0.08886657,  1.74562687],
        [ 0.16611899,  0.26145049,  1.16078869, -0.37956303],
        [ 0.57861715, -2.3339009 ,  2.31086379,  0.51286973]]),
 array([[ 1.95164769, -1.3578117 , -1.20031984],
        [ 1.96582921, -1.31783951, -0.75124818],
        [ 1.20870387, -1.34757497, -1.92128665],
        [ 1.40297331, -1.43181297, -2.11690024],
        [ 1.31355344, -1.39938784, -2.09597185],
        [ 1.12337676, -1.29845856, -1.14170738],
        [ 2.19848644, -1.30120507, -0.3370698 ],
        [ 1.1269915 , -1.33851035, 

In [59]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        self.w1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.random.randn(hidden_size)
        self.w2 = np.random.randn(hidden_size, output_size)
        self.b1 = np.random.randn(output_size)
        
        self.layers = [
            Affine(w1,b1),
            Sigmoid(),
            Affine(w2, b2)
        ]
        
        self.params = []
        for layer in self.layers:
            self.params += layer.params
    
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [61]:
net = TwoLayerNet(2,4,3)
net.predict(x)

array([[ 1.95164769, -1.3578117 , -1.20031984],
       [ 1.96582921, -1.31783951, -0.75124818],
       [ 1.20870387, -1.34757497, -1.92128665],
       [ 1.40297331, -1.43181297, -2.11690024],
       [ 1.31355344, -1.39938784, -2.09597185],
       [ 1.12337676, -1.29845856, -1.14170738],
       [ 2.19848644, -1.30120507, -0.3370698 ],
       [ 1.1269915 , -1.33851035, -1.95384532],
       [ 1.91663256, -1.30042158, -0.45500835],
       [ 2.10591097, -1.41861765, -1.33555061]])

# 1.3.1 손실 함수

In [63]:
class Softmax:
    def __init__(self):
        self.params = []
    
    def forward(self,x):
        return np.exp(x-np.max(x))/np.sum(np.exp(x))

In [64]:
softmax = Softmax()
softmax.forward(net.predict(x))

array([[0.0127388 , 0.00046542, 0.00054481],
       [0.01292074, 0.0004844 , 0.00085364],
       [0.00605999, 0.00047021, 0.00026493],
       [0.0073594 , 0.00043223, 0.00021786],
       [0.00672989, 0.00044647, 0.00022247],
       [0.00556436, 0.00049388, 0.0005777 ],
       [0.01630531, 0.00049253, 0.00129167],
       [0.00558451, 0.00047449, 0.00025645],
       [0.01230047, 0.00049292, 0.00114797],
       [0.01486361, 0.00043797, 0.0004759 ]])

In [71]:
class CrossEntrophyLoss:
    def __init__(self, batch_size):
        self.params = []
        self.batch_size = batch_size
    
    def forward(self, y, t):
        return -(np.sum(np.sum(t*np.log(y))))/self.batch_size

In [79]:
loss = CrossEntrophyLoss(2)
loss.forward(softmax.forward(net.predict(x)), np.ones_like(net.predict(x)))

99.97448063779545

# 1.3.2 미분과 기울기

### 딥러닝에서 기울기는 수학적 기울기와 달리 벡터에 대한 미분으로 한정된다.

# 1.3.3 연쇄 법칙

### 오차역전파법 (back-propagation)의 기본이 되는 이론으로, 국소적 미분값을 구하면 전체 미분값을 구할 수 있음을 시사.

# 1.3.4 계산 그래프

### 계산그래프에서 역방향에 편미분을 나타내면 순, 역방향 모두 포함하는 계산 그래프를 나타낼 수 있다.